In [ ]:
import os
from pathlib import Path
from openai import OpenAI

from segment_finder import SegmentFinder
from segment_rewriter import SegmentRewriter
from utils.pdf_utils import compile_tex_to_pdf, extract_text_from_pdf
from utils.segment_utils import inject_segment_markers, replace_marked_segment
from utils.context_utils import extract_job_terms_tfidf, match_terms, build_context

In [ ]:
resume_name = 'resume-1'
job_description_name = 'job-description-1'

input_path = Path('input')
output_path = Path('output')
output_path.mkdir(parents = True, exist_ok = True)

resume_tex_path = input_path / f'resumes/{resume_name}.tex'
resume_pdf_path = input_path / f'resumes/pdfs/{resume_name}.pdf'
job_description_path = input_path / f'jobs/{job_description_name}.txt'

In [ ]:
def read_content(path):
    return path.read_text(encoding = 'utf-8')

resume_tex_content = read_content(resume_tex_path)
job_description_content = read_content(job_description_path)

print('resume length:', len(resume_tex_content))
print('job description length:', len(job_description_content))

In [ ]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise Exception('OPENAI_API_KEY not set.')

openai_client = OpenAI(api_key = OPENAI_API_KEY)
openai_model = 'gpt-4o-mini'

In [ ]:
segment_finder = SegmentFinder(openai_client, openai_model)
segments = segment_finder.call_segment_finder(resume_tex_content, job_description_content)
marked_resume, invalid_segment_ids = inject_segment_markers(resume_tex_content, segments)

segments = [segment for segment in segments if segment.id not in invalid_segment_ids]

In [ ]:
segments

In [ ]:
resume_text_content = extract_text_from_pdf(resume_pdf_path)
resume_text_content = resume_text_content.replace('\n', ' ')

job_terms = extract_job_terms_tfidf(job_description_content, resume_text_content, top_k = 50)
present_terms, missing_terms = match_terms(resume_text_content, job_terms)
context = build_context(present_terms, missing_terms)

In [ ]:
context

In [ ]:
segment_rewriter = SegmentRewriter(openai_client, openai_model)

rewrites = []
for segment in segments:
    rewrites.append(segment_rewriter.call_segment_rewriter(segment.id, segment.latex, job_description_content, context))

In [ ]:
rewrites

In [ ]:
optimized_resume_tex = marked_resume
for rewrite in rewrites:
    optimized_resume_tex = replace_marked_segment(optimized_resume_tex, rewrite.segment_id, rewrite.rewritten_latex)

In [ ]:
optimized_dir = output_path / f'optimized-{resume_name}'
optimized_dir.mkdir(parents = True, exist_ok = True)

optimized_tex_path = optimized_dir / f'optimized-{resume_name}.tex'

optimized_tex_path.write_text(optimized_resume_tex, encoding = 'utf-8')
compile_tex_to_pdf(optimized_tex_path, output_path / f'optimized-{resume_name}')